In [98]:
import numpy as np
import pandas as pd
import os

In [99]:
df1 = pd.read_csv('../Data/Clean/USAFacts/COVIDcounts_county_clean.csv', index_col = 0)
print(df1.shape)
df1.head()

(3142, 895)


,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,5813,5818,5825,5832,5832,5832,5857,5863,5880,5891
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,6630,6648,6654,6663,6663,6663,6679,6687,6700,6704
5,1009,Blount County,AL,1,0,0,0,0,0,0,...,15246,15249,15272,15283,15283,15283,15324,15335,15351,15370


In [100]:
for index, column in enumerate(df1.columns[:]):
    if column.startswith('2020-12-30'):
        labs20 = column
    if column.startswith('2021-12-30'):
        labs21 = column
    if column.startswith('2022-06-30'):
        labs22 = column

print(labs20)
print(labs21)
print(labs22)

2020-12-30
2021-12-30
2022-06-30


In [101]:
df1 = pd.DataFrame(np.repeat(df1.values, 3, axis = 0), columns = df1.columns)
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905


In [102]:
df1['year'] = None
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index, -1] = 2020
    elif index % 3 == 1:
        df1.iloc[index, -1] = 2021
    else:
        df1.iloc[index, -1] = 2022
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2020
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2021
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2022
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2020
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2021


In [103]:
df1['covid_cases'] = None
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,None
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,None
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,None
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,None
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,None


In [104]:
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index,-1] = df1.loc[index,labs20]
    elif index % 3 == 1:
        df1.iloc[index,-1] = df1.loc[index,labs21]
    else:
        df1.iloc[index,-1] = df1.loc[index,labs22]
df1

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,4164
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,10953
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,16536
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,13392
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,39583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2021,1880
9422,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2022,2482
9423,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2020,475
9424,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2021,1254


In [105]:
df = df1[['year', 'fips','covid_cases', 'county_name']]
print(df.dtypes)
df[['year','fips','covid_cases']] = df[['year','fips','covid_cases']].apply(pd.to_numeric)
print(df.dtypes)
df

year           object
fips           object
covid_cases    object
county_name    object
dtype: object
year            int64
fips            int64
covid_cases     int64
county_name    object
dtype: object


C:\Users\FPSKINSMANC\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,year,fips,covid_cases,county_name
0,2020,1001,4164,Autauga County
1,2021,1001,10953,Autauga County
2,2022,1001,16536,Autauga County
3,2020,1003,13392,Baldwin County
4,2021,1003,39583,Baldwin County
...,...,...,...,...
9421,2021,56043,1880,Washakie County
9422,2022,56043,2482,Washakie County
9423,2020,56045,475,Weston County
9424,2021,56045,1254,Weston County


In [106]:
df.to_csv('../Data/Intermediate/COVIDcounty_cases_processed.csv', index = False)

In [107]:
dp03, dp05 = [], []
path = '../Data/Clean/ACS'
for file in os.listdir(path):
    if file.find('DP03') != -1:
        print(file)
        dp03.append(pd.read_csv(os.path.join(path, file), index_col = 0))
    if file.find('DP05') != -1:
        print(file)
        dp05.append(pd.read_csv(os.path.join(path, file), index_col = 0))

i = 0
while i != len(dp03):
    dp03[i].drop(columns = ['geographic_area_name','id'], inplace = True)
    print('03: ', dp03[i].shape)
    dp05[i].drop(columns = ['Geographic Area Name','id'], inplace = True)
    print('05: ',dp05[i].shape)
    i += 1

ACSDP032013_clean.csv
ACSDP032014_clean.csv
ACSDP032015_clean.csv
ACSDP032016_clean.csv
ACSDP032017_clean.csv
ACSDP032018_clean.csv
ACSDP032019_clean.csv
ACSDP032020_clean.csv
ACSDP052013_clean.csv
ACSDP052014_clean.csv
ACSDP052015_clean.csv
ACSDP052016_clean.csv
ACSDP052017_clean.csv
ACSDP052018_clean.csv
ACSDP052019_clean.csv
ACSDP052020_clean.csv
03:  (3221, 20)
05:  (3221, 33)
03:  (3220, 20)
05:  (3220, 33)
03:  (3220, 20)
05:  (3220, 33)
03:  (3220, 20)
05:  (3220, 33)
03:  (3220, 20)
05:  (3220, 34)
03:  (3220, 20)
05:  (3220, 34)
03:  (3220, 20)
05:  (3220, 34)
03:  (3221, 20)
05:  (3221, 34)


In [108]:
i = 2
tot_dp03 = pd.concat([dp03[0], dp03[1]], ignore_index = True)
tot_dp05 = pd.concat([dp05[0], dp05[1]], ignore_index = True)

while i != len(dp03):
    tot_dp03 = pd.concat([tot_dp03, dp03[i]], ignore_index = True)
    tot_dp05 = pd.concat([tot_dp05, dp05[i]], ignore_index = True)
    i += 1

print(tot_dp03.shape)
print(tot_dp05.shape)
tot_dp03

(25762, 28)
(25762, 34)


,per_capita_income_2013,employment_pop_ratio,pct_employment_status!!population_16_years_and_over!!in_labor_force!!civilian_labor_force!!unemployed,pct_ind_agriculture_forestry_fishing_and_hunting_and_mining,pct_ind_construction,pct_ind_manufacturing,pct_ind_wholesale_trade,pct_ind_retail_trade,pct_ind_transportation_and_warehousing_and_utilities,pct_ind_information,...,year,fips,per_capita_income_2014,per_capita_income_2015,unemployment_rate,per_capita_income_2016,per_capita_income_2017,per_capita_income_2018,per_capita_income_2019,per_capita_income_2020
0,24571.0,56.7,5.9,1.4,5.8,12.9,3.0,13.5,5.9,1.1,...,2013,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26766.0,54.8,5.6,1.9,8.9,8.9,2.9,14.6,5.1,1.3,...,2013,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16829.0,42.4,7.1,3.2,5.9,23.3,1.7,12.1,6.2,1.0,...,2013,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17427.0,43.6,6.1,2.2,9.6,19.9,3.1,14.2,6.9,0.1,...,2013,1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20730.0,50.6,5.5,2.9,9.9,17.1,3.6,11.3,7.6,0.9,...,2013,1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25757,NaN,35.1,NaN,1.0,6.2,12.5,4.2,11.0,3.9,1.1,...,2020,72145,NaN,NaN,18.1,NaN,NaN,NaN,NaN,11444.0
25758,NaN,29.0,NaN,0.7,18.4,2.6,0.0,1.0,5.4,0.0,...,2020,72147,NaN,NaN,13.1,NaN,NaN,NaN,NaN,9096.0
25759,NaN,36.1,NaN,3.9,7.1,17.7,0.4,10.1,2.9,0.5,...,2020,72149,NaN,NaN,20.0,NaN,NaN,NaN,NaN,11596.0
25760,NaN,31.2,NaN,0.6,5.1,19.6,1.0,11.7,3.4,0.8,...,2020,72151,NaN,NaN,18.1,NaN,NaN,NaN,NaN,10124.0


In [109]:
tot_dp05.columns

Index(['pct_sex_age_under_5', 'pct_sex_age_5_to_9', 'pct_sex_age_10_to_14',
       'pct_sex_age_15_to_19', 'pct_sex_age_20_to_24', 'pct_sex_age_25_to_34',
       'pct_sex_age_35_to_44', 'pct_sex_age_45_to_54', 'pct_sex_age_55_to_59',
       'pct_sex_age_60_to_64', 'pct_sex_age_65_to_74', 'pct_sex_age_75_to_84',
       'pct_sex_age_85+', 'median_age', 'pct_sex_age_18+', 'pct_sex_age_21+',
       'pct_sex_age_62+', 'pct_sex_age_65+', 'pct_sex_age_18+_male',
       'pct_sex_age_18+_female', 'pct_sex_age_65+_male',
       'pct_sex_age_65+_female', 'pct_one_or_more_race',
       'pct_one_or_more_race_white', 'pct_one_or_more_race_black',
       'pct_one_or_more_race_native_american', 'pct_one_or_more_race_asian',
       'pct_one_or_more_race_native_hawaiian_pacific_islander',
       'pct_one_or_more_race_other_race', 'pct_ethnicity_hispanic', 'year',
       'total_pop', 'fips', 'pct_sex_age_16+'],
      dtype='object')

In [110]:
tot_dp05

,pct_sex_age_under_5,pct_sex_age_5_to_9,pct_sex_age_10_to_14,pct_sex_age_15_to_19,pct_sex_age_20_to_24,pct_sex_age_25_to_34,pct_sex_age_35_to_44,pct_sex_age_45_to_54,pct_sex_age_55_to_59,pct_sex_age_60_to_64,...,pct_one_or_more_race_black,pct_one_or_more_race_native_american,pct_one_or_more_race_asian,pct_one_or_more_race_native_hawaiian_pacific_islander,pct_one_or_more_race_other_race,pct_ethnicity_hispanic,year,total_pop,fips,pct_sex_age_16+
0,6.4,7.1,8.0,7.6,5.9,11.9,14.6,15.1,6.0,4.8,...,18.8,0.9,1.3,0.0,0.8,2.5,2013,54907,1001,NaN
1,6.0,6.6,6.2,6.3,5.4,11.5,12.7,14.4,6.7,6.9,...,9.9,1.3,1.1,0.0,1.5,4.4,2013,187114,1003,NaN
2,5.8,6.3,5.8,5.8,6.7,13.9,13.4,14.2,7.1,5.9,...,47.5,1.4,0.7,0.0,3.7,4.9,2013,27321,1005,NaN
3,5.7,6.5,6.0,7.2,7.3,11.7,15.1,15.2,5.3,6.5,...,22.5,0.7,0.2,0.0,0.0,1.9,2013,22754,1007,NaN
4,6.3,6.4,7.2,6.9,5.7,11.7,13.4,14.0,6.0,7.0,...,2.0,1.2,0.3,0.1,1.2,8.3,2013,57623,1009,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25757,4.3,5.0,5.9,6.5,6.7,12.7,12.2,13.1,6.5,6.9,...,5.8,0.1,0.0,0.0,20.7,97.1,2020,51089,72145,83.7
25758,5.1,5.5,4.8,6.8,7.3,9.0,12.9,11.0,9.5,5.9,...,12.5,0.1,0.2,0.0,44.1,93.6,2020,8508,72147,83.3
25759,4.8,5.8,5.9,7.4,7.4,12.3,11.1,13.1,7.7,6.8,...,29.8,0.2,0.0,0.0,15.8,99.8,2020,21899,72149,82.3
25760,3.6,5.2,5.8,6.5,6.7,11.6,11.7,13.4,7.2,7.4,...,68.8,0.1,0.0,0.0,17.8,99.9,2020,32867,72151,84.2


In [111]:
tot_dp03.to_csv('../Data/Intermediate/ACSDP03_merge.csv')
tot_dp05.to_csv('../Data/Intermediate/ACSDP05_merge.csv')

In [112]:
tot_dp03.shape

(25762, 28)

In [113]:
hps = pd.read_csv('../Data/Clean/HPS/hps_clean.csv', index_col = 0)
print(hps.columns)

Index(['year', 'area', 'pct_delayed_healthcare'], dtype='object')


In [114]:
hps['state_fips'] = None
state = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 
         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 
         38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56]
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming']

dic = {state[i]: us_states[i] for i in range(len(state))}

print(len(state))
print(len(us_states))
print(dic)

hps['state_fips'] = dic
hps

51
51
{1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming'}


,year,area,pct_delayed_healthcare,state_fips
0,2020,Alabama,39.200000,1
1,2020,Alaska,45.708333,2
2,2020,Arizona,41.016667,4
3,2020,Arkansas,40.575000,5
4,2020,California,41.116667,6
5,2020,Colorado,41.158333,8
6,2020,Connecticut,41.391667,9
7,2020,Delaware,38.766667,10
8,2020,District of Columbia,43.500000,11
9,2020,Florida,40.216667,12


In [115]:
hps.to_csv('../Data/Intermediate/hps_processed.csv')